# AgenticART Dojo - Fine-Tune WhiteRabbitNeo\n\nThis notebook fine-tunes WhiteRabbitNeo on ADB command data collected from the Dojo.\n\n**Requirements:** T4 GPU (free tier) or better

In [ ]:
# Install dependencies\n!pip install -q unsloth\n!pip install -q --no-deps trl peft accelerate bitsandbytes

In [ ]:
# Upload your training data\nfrom google.colab import files\nuploaded = files.upload()  # Upload training_data.json

In [ ]:
import json\nfrom datasets import Dataset\nfrom unsloth import FastLanguageModel\nfrom trl import SFTTrainer\nfrom transformers import TrainingArguments\n\n# Load training data\nwith open('training_data.json') as f:\n    training_data = json.load(f)\nprint(f'Loaded {len(training_data)} examples')

In [ ]:
# Format data\ndef format_alpaca(example):\n    if example.get('input'):\n        text = f\"\"\"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}\"\"\"\n    else:\n        text = f\"\"\"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}\"\"\"\n    return {'text': text}\n\ndataset = Dataset.from_list(training_data)\ndataset = dataset.map(format_alpaca)\nprint(f'Formatted {len(dataset)} examples')

In [ ]:
# Load model\nmodel, tokenizer = FastLanguageModel.from_pretrained(\n    model_name='WhiteRabbitNeo/WhiteRabbitNeo-2.5-Qwen-2.5-Coder-7B',\n    max_seq_length=2048,\n    load_in_4bit=True,\n)\n\n# Add LoRA\nmodel = FastLanguageModel.get_peft_model(\n    model,\n    r=16,\n    lora_alpha=32,\n    lora_dropout=0.05,\n    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],\n    use_gradient_checkpointing='unsloth',\n)

In [ ]:
# Training\ntrainer = SFTTrainer(\n    model=model,\n    tokenizer=tokenizer,\n    train_dataset=dataset,\n    args=TrainingArguments(\n        output_dir='./output',\n        num_train_epochs=3,\n        per_device_train_batch_size=4,\n        gradient_accumulation_steps=4,\n        learning_rate=1e-05,\n        warmup_ratio=0.03,\n        fp16=True,\n        logging_steps=10,\n        optim='adamw_8bit',\n    ),\n    dataset_text_field='text',\n    max_seq_length=2048,\n    packing=True,\n)\n\ntrainer.train()

In [ ]:
# Save as GGUF for Ollama\nmodel.save_pretrained_gguf(\n    'WhiteRabbitNeo-ADB-Dojo',\n    tokenizer,\n    quantization_method='q4_k_m',\n)\n\n# Download the model\nfrom google.colab import files\nfiles.download('WhiteRabbitNeo-ADB-Dojo-unsloth.Q4_K_M.gguf')

## Import to Ollama\n\nAfter downloading, create a Modelfile:\n```\nFROM ./WhiteRabbitNeo-ADB-Dojo-unsloth.Q4_K_M.gguf\n```\n\nThen run:\n```bash\nollama create whiterabbit-adb-dojo -f Modelfile\n```